# Processing of gatk vcf

In [1]:
import pandas as pd
import numpy as np
import io
import os
import gzip
import sys

### Plan:
1. Variant calling
2. Filtering
3. Annotation

# 1. Variant calling

```
Readjusting RG in bams
java -jar picard.jar AddOrReplaceReadGroups I=${DIR}/bam_files/B10.sorted.calmd.bam O=${DIR}/bam_files_RG/B10_rg.bam RGID=HNVJGDSX2.4 RGLB=B10 RGPL=ILLUMINA RGPU=HNVJGDSX2.4.B10 RGSM=B10
Running HaplotypeCaller
gatk HaplotypeCaller -R ${DIR}/reference_files/S288C_reference_sequence_R64-3-1_20210421_chr.fasta -I ${DIR}/bam_files_RG/B10_rg.bam -O HaplotypeCaller/B10.g.vcf.gz -ploidy 1 -ERC GVCF --min-base-quality-score 20 --annotation StrandBiasBySample
Running CombineGVCFs

gatk CombineGVCFs -R $REF hapcalls_NC02.txt \
 -O CombineGenotypes/set_NC02.g.vcf \
 -G StandardAnnotation
Running GenotypeGVCFs

gatk GenotypeGVCFs -R $REF \
-V CombineGenotypes/set_LL13.g.vcf \
-O CombineGenotypes/set_LL13.vcf.gz \
-G StandardAnnotation \
--annotation Coverage \
--annotation QualByDepth \
--annotation MappingQuality \
--annotation MappingQualityRankSumTest \
--annotation ExcessHet
Split to biallelic SNPs and INDELs

gatk SelectVariants -R $REF \
-V CombineGenotypes/set_LL13.vcf.gz \
-O CombineGenotypes/set_LL13_SNPs.vcf.gz \
--restrict-alleles-to BIALLELIC \
--exclude-filtered true \
--exclude-non-variants true \
--remove-unused-alternates true \
--select-type SNP
VariantFiltration for SNPs

gatk VariantFiltration -R $REF \
-V CombineGenotypes/set_${set}_SNPs.vcf.gz \
-O CombineGenotypes/set_${set}_SNPs_VF.vcf.gz \
-filter "vc.hasAttribute('QD') && QD < 2.0" \
--filter-name "QD" \
-filter "vc.hasAttribute('QUAL') && QUAL < 30.0" \
--filter-name "QUAL" \
-filter "vc.hasAttribute('MQ') && MQ < 40.0" \
--filter-name "MapQ" \
-filter "vc.hasAttribute('FS') && FS > 60.0" \
--filter-name "FS" \
-filter "vc.hasAttribute('FS') && SOR > 4.0" \
--filter-name "SOR" \
-filter "vc.hasAttribute('MQRankSum') && MQRankSum < -12.5" \
--filter-name "MQRS" \
-filter "vc.hasAttribute('ReadPosRankSum') && ReadPosRankSum < -8.0" \
--filter-name "RPRS" \
--genotype-filter-expression "DP < 4" \
--genotype-filter-name "DP" \
--set-filtered-genotype-to-no-call true \
--QUIET true
VariantFiltration for INDELs

gatk VariantFiltration -R $REF \
-V CombineGenotypes/set_${set}_INDELs.vcf.gz \
-O CombineGenotypes/set_${set}_INDELs_VF.vcf.gz \
-filter "vc.hasAttribute('QD') && QD < 2.0" \
--filter-name "QD" \
-filter "vc.hasAttribute('QUAL') && QUAL < 30.0" \
--filter-name "QUAL" \
-filter "vc.hasAttribute('FS') && FS > 200.0" \
--filter-name "FS" \
-filter "vc.hasAttribute('ReadPosRankSum') && ReadPosRankSum < -20.0" \
--filter-name "RPRS" \
--genotype-filter-expression "DP < 4" \
--genotype-filter-name "DP" \
--set-filtered-genotype-to-no-call true \
--QUIET true
Selecting variants

gatk SelectVariants -R $REF \
-V CombineGenotypes/set_${set}_SNPs_VF.vcf.gz \
-O CombineGenotypes/set_${set}_SNPs_SV.vcf.gz \
--exclude-filtered true \
--exclude-non-variants true \
--remove-unused-alternates true \
```

# 2. Filtering

```
TMP=../DATA_temp

for bg in LL13 NC02
  do
  echo $bg

  for type in SNPs INDELs
      do

      # Sample filters
      # 1. Masking genotypes with DP < 4 (mask) and GT 2 or 3; 
      # 2. Allelic depth (AD) of the second most common variant within genotype is less than 4
      # 3. Allelic depth of the second most frequent variant to most frequent variant is less than 0.2

      python filterAD2_gatk.py ../DATA/set_${bg}_${type}_SV.vcf.gz
      mv ../DATA/set_${bg}_${type}_SV_filterAD.vcf $TMP/set_${bg}_${type}_filterAD.vcf
      bgzip -f $TMP/set_${bg}_${type}_filterAD.vcf .
      tabix -f -p vcf $TMP/set_${bg}_${type}_filterAD.vcf.gz

      # SNP filters
      # 1. Mean per sample coverge is higher than 10
      # 2. Max total DP is less than 20000
      # 3. SNP QUAL is higher than 20
      # 4. Mapping quality MQ is higher than 40
      # 5. Fixed alleles removed (AF>0.99 or AC==0)

      bcftools filter -e "AVG(FORMAT/DP)<10 || INFO/DP>20000 || QUAL<20 || MQ<40 ||  (AF > 0.99) || (AC == 0)" -Ou $TMP/set_${bg}_${type}_filterAD_tags.vcf.gz | bcftools view -m2 -Oz -o $TMP/set_${bg}_${type}_FLT1.vcf.gz -
      bcftools stats $TMP/snp_bcftools_${bg}_FLT1.vcf.gz > snp_bcftools_${bg}_FLT1.stats
      tabix -f -p vcf $TMP/snp_bcftools_${bg}_FLT1.vcf.gz
      done
  done
```

Python script available here: [filterAD2_gatk.py](filterAD2_gatk.py)

# 3. Annotation

#### Annotating with snpEff and *S. cerevisiae* genome vR64-3-1 and splitting into SNPs and INDELs
```
conda activate snpeff-5.0
# SnpEff version SnpEff 5.0e (build 2021-03-09 06:01), by Pablo Cingolani
snpEff_DIR=/home/anna/soft/snpEff
TMP=../DATA_temp
CONFIG=/home/software/snpEff/snpEff.config

for bg in LL13 NC02
  do

  for type in SNPs INDELs
    do

    # SnpEff
    snpEff -c $CONFIG R64.3.1 $TMP/set_${bg}_${type}_FLT1.vcf.gz > $TMP/set_${bg}_${type}_snpEff.vcf
    bgzip -f $TMP/set_${bg}_${type}_snpEff.vcf
    tabix -f -p vcf $TMP/set_${bg}_${type}_snpEff.vcf.gz
    cp $TMP/set_${bg}_${type}_snpEff.vcf* ../DATA/
    done
  done
```

# 3. Getting a summary table

In [2]:
important_categories = ['synonymous_variant','stop_gained','missense_variant']

#### Function for reading vcf

In [3]:
def readVcfGzip(filepath):
    fh = gzip.open(filepath,'rt')
    lines = [l for l in fh if not l.startswith('##')]
    df_vcf = pd.read_csv(io.StringIO(''.join(lines)),sep='\t')
    return(df_vcf)

#### Function for formatting vcf into stacked table

In [4]:
def stackMutations(vcf_file):
    
    # Vcf to dataframe
    df_vcf = readVcfGzip(vcf_file)
    
    # Formatting snpeff annotations & mutation
    infos = df_vcf['INFO'].values.tolist()
    I, G = [],[]
    #T = []
    for info in infos:
        muts = []
        ninfo = [i.split('=')[1] for i in info.split(';') if i.startswith('ANN=')][0]
        sinfo = [i for i in ninfo.split(',')]
        #T.append(ninfo)
        mut = ":".join([sinfo[0].split('|')[0],sinfo[0].split('|')[1]]), sinfo[0].split('|')[3].replace("_CDS","")
        muts.append(mut)
        for gene in sinfo[1:]:
            ginfo = gene.split('|')
            mut = ":".join([ginfo[0],ginfo[1]]), ginfo[3].replace("_CDS","")
            if (ginfo[1] in important_categories) & (mut not in muts):
                muts.append(mut)
        all_muts = '/'.join([m[0] for m in muts])
        all_genes = '/'.join([m[1] for m in muts])
        I.append(all_muts)
        G.append(all_genes)
    df_vcf['MUT'] = I
    df_vcf['GENE'] = G
    df_vcf['mutID'] = df_vcf.apply(lambda x: x["#CHROM"]+":"+str(x["POS"])+":"+str(x["REF"])+":"+x["MUT"], axis=1)
    df_vcf.head()
    
    # Formatting samples
    df_vcf_ = df_vcf.drop(columns=["ID","FILTER","INFO","FORMAT","MUT"])
    df_stack = df_vcf_.set_index(["#CHROM","POS","REF","ALT","QUAL","GENE","mutID"]).stack().reset_index().rename(columns={"level_7":"sampleID",0:"GT"})
    del df_vcf_
    df_stack['sample'] = df_stack['sampleID'].apply(lambda x: x.split('/')[-1].split('.')[0])
    df_stack['snp'] = df_stack['GT'].apply(lambda x: 1 if x.split(':')[0] == '1' else 0)
    df_stack = df_stack.drop(columns=["REF","ALT","QUAL","sampleID","GT"])
    df_stack = df_stack[df_stack['snp']==1].reset_index(drop=True)
    del df_vcf
    return(df_stack)

In [5]:
def stackMutationsWithAnn(vcf_file):
    
    # Vcf to dataframe
    df_vcf = readVcfGzip(vcf_file)
    
    # Formatting snpeff annotations & mutation
    infos = df_vcf['INFO'].values.tolist()
    I, G = [],[]
    #T = []
    for info in infos:
        muts = []
        ninfo = [i.split('=')[1] for i in info.split(';') if i.startswith('ANN=')][0]
        sinfo = [i for i in ninfo.split(',')]
        #T.append(ninfo)
        mut = ":".join([sinfo[0].split('|')[0],sinfo[0].split('|')[1]]), sinfo[0].split('|')[3].replace("_CDS","")
        muts.append(mut)
        for gene in sinfo[1:]:
            ginfo = gene.split('|')
            mut = ":".join([ginfo[0],ginfo[1]]), ginfo[3].replace("_CDS","")
            if (ginfo[1] in important_categories) & (mut not in muts):
                muts.append(mut)
        all_muts = '/'.join([m[0] for m in muts])
        all_genes = '/'.join([m[1] for m in muts])
        I.append(all_muts)
        G.append(all_genes)
        
    A = []
    ANN = []
    for info in infos:
        muts = []
        ninfo = [i.split('=')[1] for i in info.split(';') if i.startswith('ANN=')][0]
        ANN.append(ninfo)
        sinfo = [i for i in ninfo.split(',')]
        #mut = ":".join([sinfo[0].split('|')[0],sinfo[0].split('|')[1]]), sinfo[0].split('|')[3].replace("_CDS","")
        #muts.append(mut)
        for gene in sinfo:
            ginfo = gene.split('|')
            mut = ":".join([ginfo[0],ginfo[1],ginfo[3].replace("_CDS","")])
            if mut not in muts:
                muts.append(mut)
        all_muts = '/'.join(muts)
        A.append(all_muts)
    
    df_vcf['MUT'] = I
    df_vcf['GENE'] = G
    df_vcf['MUT_ALL'] = A
    df_vcf['ANN'] = ANN
    df_vcf['mutID'] = df_vcf.apply(lambda x: x["#CHROM"]+":"+str(x["POS"])+":"+str(x["REF"])+":"+x["MUT"], axis=1)
    df_vcf.head()
    
    # Formatting samples
    df_vcf_ = df_vcf.drop(columns=["ID","FILTER","INFO","FORMAT","MUT"])
    df_stack = df_vcf_.set_index(["#CHROM","POS","REF","ALT","QUAL","GENE","mutID","MUT_ALL","ANN"]).stack().reset_index().rename(columns={"level_9":"sampleID",0:"GT"})
    del df_vcf_
    df_stack['sample'] = df_stack['sampleID'].apply(lambda x: x.split('/')[-1].split('.')[0])
    df_stack['snp'] = df_stack['GT'].apply(lambda x: 1 if x.split(':')[0] == '1' else 0)
    df_stack = df_stack.drop(columns=["QUAL","sampleID"])
    df_stack = df_stack[df_stack['snp']==1].reset_index(drop=True)
    del df_vcf
    return(df_stack)

# SNPs

#### Combining two backgrounds together

In [6]:
vcf1 = "../DATA/set_LL13_SNPs_snpEff.vcf.gz"
vcf2 = "../DATA/set_NC02_SNPs_snpEff.vcf.gz"
# LL13
df_vcf1 = stackMutationsWithAnn(vcf1)
df_vcf1['background'] = "LL13-040"
parent_variants_LL13 = df_vcf1.loc[df_vcf1['sample'].isin(['H13']), ['#CHROM','POS']].reset_index(drop=True)
df_vcf1_merged = pd.merge(df_vcf1, parent_variants_LL13, on = ['#CHROM','POS'], how = 'outer', indicator=True)
df_vcf1_noWT = df_vcf1_merged[df_vcf1_merged['_merge'] == 'left_only'].reset_index(drop=True)
# NC02
df_vcf2 = stackMutationsWithAnn(vcf2)
df_vcf2['background'] = "NC-02"
parent_variants_NC02 = df_vcf2.loc[df_vcf2['sample'].isin(['E23']), ['#CHROM','POS']].reset_index(drop=True)
df_vcf2_merged = pd.merge(df_vcf2, parent_variants_NC02, on = ['#CHROM','POS'], how = 'outer', indicator=True)
df_vcf2_noWT = df_vcf2_merged[df_vcf2_merged['_merge'] == 'left_only'].reset_index(drop=True)

# Merged
df = pd.concat([df_vcf1_noWT, df_vcf2_noWT],ignore_index=True)
print(df.shape)
df.head()

(27061, 13)


,#CHROM,POS,REF,ALT,GENE,mutID,MUT_ALL,ANN,GT,sample,snp,background,_merge
0,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,23:23:PASS:99:995,0",B10,1,LL13-040,left_only
1,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,22:22:PASS:99:974,0",B13,1,LL13-040,left_only
2,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,60:60:PASS:99:2628,0",B4,1,LL13-040,left_only
3,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,55:55:PASS:99:2324,0",B5,1,LL13-040,left_only
4,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,46:46:PASS:99:1907,0",B6,1,LL13-040,left_only


In [7]:
df['NGENE'] = df['GENE'].apply(lambda x: list(set(x.split('/')))[0] if len(set(x.split('/'))) == 1 else x)
#df[df['GENE'] == "YHR128W/YHR128W"].head()

#### Getting gene descriptions from SGD

#### Reading gene names extracted from SGD for corresponding gene IDs 

In [8]:
dg = pd.read_csv('../../DATA/S288C_reference_genome_R64-3-1_20210421/gene_list_SGD.tsv',sep="\t",names=['ID','Gene','Species','Symbol','Name'])
dg.head()

,ID,Gene,Species,Symbol,Name
0,S000000001,YAL001C,S. cerevisiae,TFC3,Transcription Factor class C
1,S000000002,YAL002W,S. cerevisiae,VPS8,Vacuolar Protein Sorting
2,S000000003,YAL003W,S. cerevisiae,EFB1,Elongation Factor Beta
3,S000000004,YAL005C,S. cerevisiae,SSA1,Stress-Seventy subfamily A
4,S000000005,YAL007C,S. cerevisiae,ERP2,Emp24p/Erv25p Related Protein


#### Adding missing gene IDs

In [9]:
dictG = {ele:np.array(ele.split('/')) for ele in df['NGENE'].drop_duplicates().values}
all_gene_ids = np.concatenate(list(dictG.values()))
missing_IDs = [ele for ele in all_gene_ids if ele not in dg['Gene'].values]
missing_dg = pd.DataFrame({"ID":"SXX","Gene":missing_IDs,"Species":"S. cerevisiae","Symbol":"UNK","Name":"UNK"})
dg_combined = pd.concat([dg, missing_dg], ignore_index = True)

gene_symbol = {a:b for a,b in dg_combined.get(['Gene','Symbol']).fillna('UNK').values.tolist()}
gene_name = {a:b for a,b in dg_combined.get(['Gene','Name']).fillna('UNK').values.tolist()}

df['GENE_SYMBOL'] = df['NGENE'].apply(lambda x: '/'.join([gene_symbol[i] for i in x.split('/')]))
df['GENE_NAME'] = df['NGENE'].apply(lambda x: '/'.join([gene_name[i] for i in x.split('/')]))
print(df.shape)
df.to_csv('gatk_variants_SNP.tab',sep='\t',header=True,index=True)
df.head()

(27061, 16)


,#CHROM,POS,REF,ALT,GENE,mutID,MUT_ALL,ANN,GT,sample,snp,background,_merge,NGENE,GENE_SYMBOL,GENE_NAME
0,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,23:23:PASS:99:995,0",B10,1,LL13-040,left_only,YAL069W,UNK,UNK
1,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,22:22:PASS:99:974,0",B13,1,LL13-040,left_only,YAL069W,UNK,UNK
2,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,60:60:PASS:99:2628,0",B4,1,LL13-040,left_only,YAL069W,UNK,UNK
3,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,55:55:PASS:99:2324,0",B5,1,LL13-040,left_only,YAL069W,UNK,UNK
4,chrI,353,C,T,YAL069W,chrI:353:C:T:missense_variant,T:missense_variant:YAL069W/T:upstream_gene_var...,T|missense_variant|MODERATE|YAL069W_CDS|GENE_Y...,"1:0,46:46:PASS:99:1907,0",B6,1,LL13-040,left_only,YAL069W,UNK,UNK


#### Grouping by snp - how many samples

In [10]:
dsamp = df.groupby(["#CHROM","POS"]).agg(N_sample = ("sample","count")).reset_index()
dsamp_low = dsamp.loc[dsamp["N_sample"] < 20, ["#CHROM","POS"]]
dff = pd.merge(dsamp_low, df, on = ["#CHROM","POS"], how = "left")
print(dff.shape)
dff.head()

(5037, 16)


,#CHROM,POS,REF,ALT,GENE,mutID,MUT_ALL,ANN,GT,sample,snp,background,_merge,NGENE,GENE_SYMBOL,GENE_NAME
0,chrI,1026,C,T,YAL067W-A,chrI:1026:C:T:upstream_gene_variant,T:upstream_gene_variant:YAL067W-A/T:upstream_g...,T|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:0,15:15:PASS:99:555,0",E7,1,LL13-040,left_only,YAL067W-A,UNK,UNK
1,chrI,1026,C,T,YAL067W-A,chrI:1026:C:T:upstream_gene_variant,T:upstream_gene_variant:YAL067W-A/T:upstream_g...,T|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:1,5:6:PASS:99:122,0",F4,1,LL13-040,left_only,YAL067W-A,UNK,UNK
2,chrI,1026,C,T,YAL067W-A,chrI:1026:C:T:upstream_gene_variant,T:upstream_gene_variant:YAL067W-A/T:upstream_g...,T|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:1,5:6:PASS:99:117,0",J3,1,LL13-040,left_only,YAL067W-A,UNK,UNK
3,chrI,1232,T,C,YAL067W-A,chrI:1232:T:C:upstream_gene_variant,C:upstream_gene_variant:YAL067W-A/C:upstream_g...,C|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:2,11:13:PASS:99:405,0",B16,1,NC-02,left_only,YAL067W-A,UNK,UNK
4,chrI,1232,T,C,YAL067W-A,chrI:1232:T:C:upstream_gene_variant,C:upstream_gene_variant:YAL067W-A/C:upstream_g...,C|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:1,10:11:PASS:99:425,0",B20,1,NC-02,left_only,YAL067W-A,UNK,UNK


# INDELs

#### Combining two backgrounds

In [15]:
vcf1 = "../DATA/set_LL13_INDELs_snpEff.vcf.gz"
vcf2 = "../DATA/set_NC02_INDELs_snpEff.vcf.gz"
# LL13
df_vcf1 = stackMutationsWithAnn(vcf1)
df_vcf1['background'] = "LL13-040"
parent_variants_LL13 = df_vcf1.loc[df_vcf1['sample'].isin(['H13']), ['#CHROM','POS']].reset_index(drop=True)
df_vcf1_merged = pd.merge(df_vcf1, parent_variants_LL13, on = ['#CHROM','POS'], how = 'outer', indicator=True)
df_vcf1_noWT = df_vcf1_merged[df_vcf1_merged['_merge'] == 'left_only'].reset_index(drop=True)
# NC02
df_vcf2 = stackMutationsWithAnn(vcf2)
df_vcf2['background'] = "NC-02"
parent_variants_NC02 = df_vcf2.loc[df_vcf2['sample'].isin(['E23']), ['#CHROM','POS']].reset_index(drop=True)
df_vcf2_merged = pd.merge(df_vcf2, parent_variants_NC02, on = ['#CHROM','POS'], how = 'outer', indicator=True)
df_vcf2_noWT = df_vcf2_merged[df_vcf2_merged['_merge'] == 'left_only'].reset_index(drop=True)

# Merged
df = pd.concat([df_vcf1_noWT, df_vcf2_noWT],ignore_index=True)
print(df.shape)

# Gene name
df['NGENE'] = df['GENE'].apply(lambda x: list(set(x.split('/')))[0] if len(set(x.split('/'))) == 1 else x)
df.head()

(24216, 13)


,#CHROM,POS,REF,ALT,GENE,mutID,MUT_ALL,ANN,GT,sample,snp,background,_merge,NGENE
0,chrI,351,C,CATATAT,YAL069W,chrI:351:C:CATATAT:conservative_inframe_insertion,CATATAT:conservative_inframe_insertion:YAL069W...,CATATAT|conservative_inframe_insertion|MODERAT...,"1:1,16:17:PASS:99:594,0",E6,1,LL13-040,left_only,YAL069W
1,chrI,1400,CG,C,YAL067W-A,chrI:1400:CG:C:upstream_gene_variant,C:upstream_gene_variant:YAL067W-A/C:upstream_g...,C|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:0,5:5:PASS:99:225,0",O3,1,LL13-040,left_only,YAL067W-A
2,chrI,1482,AC,A,YAL067W-A,chrI:1482:AC:A:upstream_gene_variant,A:upstream_gene_variant:YAL067W-A/A:upstream_g...,A|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:0,17:17:PASS:99:756,0",B4,1,LL13-040,left_only,YAL067W-A
3,chrI,2232,AT,A,YAL068C,chrI:2232:AT:A:upstream_gene_variant,A:upstream_gene_variant:YAL068C/A:upstream_gen...,A|upstream_gene_variant|MODIFIER|YAL068C_CDS|G...,"1:1,18:19:PASS:99:765,0",B7,1,LL13-040,left_only,YAL068C
4,chrI,2232,AT,A,YAL068C,chrI:2232:AT:A:upstream_gene_variant,A:upstream_gene_variant:YAL068C/A:upstream_gen...,A|upstream_gene_variant|MODIFIER|YAL068C_CDS|G...,"1:3,18:21:PASS:99:675,0",C3,1,LL13-040,left_only,YAL068C


In [16]:
df[["#CHROM","POS"]].drop_duplicates()
df[df['NGENE'] == "YHR128W"]

,#CHROM,POS,REF,ALT,GENE,mutID,MUT_ALL,ANN,GT,sample,snp,background,_merge,NGENE
6040,chrVIII,362176,TG,T,YHR128W,chrVIII:362176:TG:T:frameshift_variant,T:frameshift_variant:YHR128W/T:upstream_gene_v...,T|frameshift_variant|HIGH|YHR128W_CDS|GENE_YHR...,"1:0,24:24:PASS:99:947,0",B11,1,LL13-040,left_only,YHR128W
6041,chrVIII,362273,T,TCTACCTGTGCAAAAGCAAATTGTGGAAACTGACACCAA,YHR128W,chrVIII:362273:T:TCTACCTGTGCAAAAGCAAATTGTGGAAA...,TCTACCTGTGCAAAAGCAAATTGTGGAAACTGACACCAA:frames...,TCTACCTGTGCAAAAGCAAATTGTGGAAACTGACACCAA|frames...,"1:0,41:41:PASS:99:1828,0",K7,1,LL13-040,left_only,YHR128W
6042,chrVIII,362550,GC,G,YHR128W,chrVIII:362550:GC:G:frameshift_variant,G:frameshift_variant:YHR128W/G:upstream_gene_v...,G|frameshift_variant|HIGH|YHR128W_CDS|GENE_YHR...,"1:0,14:14:PASS:99:567,0",F11,1,LL13-040,left_only,YHR128W
6043,chrVIII,362661,CCAGAGGTCAGAATTGTTACTGGTGCCCTCGA,C,YHR128W,chrVIII:362661:CCAGAGGTCAGAATTGTTACTGGTGCCCTCG...,C:frameshift_variant:YHR128W/C:upstream_gene_v...,C|frameshift_variant|HIGH|YHR128W_CDS|GENE_YHR...,"1:0,34:34:PASS:99:1535,0",E13,1,LL13-040,left_only,YHR128W
6044,chrVIII,362725,CA,C,YHR128W,chrVIII:362725:CA:C:frameshift_variant,C:frameshift_variant:YHR128W/C:upstream_gene_v...,C|frameshift_variant|HIGH|YHR128W_CDS|GENE_YHR...,"1:0,7:7:PASS:99:283,0",B13,1,LL13-040,left_only,YHR128W
17755,chrVIII,362246,ATTGT,A,YHR128W,chrVIII:362246:ATTGT:A:frameshift_variant,A:frameshift_variant:YHR128W/A:upstream_gene_v...,A|frameshift_variant|HIGH|YHR128W_CDS|GENE_YHR...,"1:1,23:24:99:990,0",J15,1,NC-02,left_only,YHR128W
17756,chrVIII,362254,TTGA,T,YHR128W,chrVIII:362254:TTGA:T:disruptive_inframe_deletion,T:disruptive_inframe_deletion:YHR128W/T:upstre...,T|disruptive_inframe_deletion|MODERATE|YHR128W...,"1:1,23:24:99:990,0",J15,1,NC-02,left_only,YHR128W
17757,chrVIII,362282,GC,G,YHR128W,chrVIII:362282:GC:G:frameshift_variant,G:frameshift_variant:YHR128W/G:upstream_gene_v...,G|frameshift_variant|HIGH|YHR128W_CDS|GENE_YHR...,"1:0,29:29:PASS:99:1143,0",K19,1,NC-02,left_only,YHR128W
17758,chrVIII,362360,CATTGTCAGAGCTGGT,C,YHR128W,chrVIII:362360:CATTGTCAGAGCTGGT:C:conservative...,C:conservative_inframe_deletion:YHR128W/C:upst...,C|conservative_inframe_deletion|MODERATE|YHR12...,"1:0,29:29:PASS:99:1314,0",B20,1,NC-02,left_only,YHR128W
17759,chrVIII,362378,ATCGATGGAGCAAGGATTAAGAGACTGTTGTAGGTCTGTGCGTATCGGT,A,YHR128W,chrVIII:362378:ATCGATGGAGCAAGGATTAAGAGACTGTTGT...,A:conservative_inframe_deletion:YHR128W/A:upst...,A|conservative_inframe_deletion|MODERATE|YHR12...,"1:0,29:29:PASS:99:1317,0",B20,1,NC-02,left_only,YHR128W


#### SGD gene names

In [17]:
dg = pd.read_csv('../../DATA/S288C_reference_genome_R64-3-1_20210421/gene_list_SGD.tsv',sep="\t",names=['ID','Gene','Species','Symbol','Name'])
dictG = {ele:np.array(ele.split('/')) for ele in df['NGENE'].drop_duplicates().values}
all_gene_ids = np.concatenate(list(dictG.values()))
missing_IDs = [ele for ele in all_gene_ids if ele not in dg['Gene'].values]
missing_dg = pd.DataFrame({"ID":"SXX","Gene":missing_IDs,"Species":"S. cerevisiae","Symbol":"UNK","Name":"UNK"})
dg_combined = pd.concat([dg, missing_dg], ignore_index = True)

gene_symbol = {a:b for a,b in dg_combined.get(['Gene','Symbol']).fillna('UNK').values.tolist()}
gene_name = {a:b for a,b in dg_combined.get(['Gene','Name']).fillna('UNK').values.tolist()}

df['GENE_SYMBOL'] = df['NGENE'].apply(lambda x: '/'.join([gene_symbol[i] for i in x.split('/')]))
df['GENE_NAME'] = df['NGENE'].apply(lambda x: '/'.join([gene_name[i] for i in x.split('/')]))
print(df.shape)
df.to_csv('gatk_variants_INDEL.tab',sep='\t',header=True,index=True)
df.head()

(24216, 16)


,#CHROM,POS,REF,ALT,GENE,mutID,MUT_ALL,ANN,GT,sample,snp,background,_merge,NGENE,GENE_SYMBOL,GENE_NAME
0,chrI,351,C,CATATAT,YAL069W,chrI:351:C:CATATAT:conservative_inframe_insertion,CATATAT:conservative_inframe_insertion:YAL069W...,CATATAT|conservative_inframe_insertion|MODERAT...,"1:1,16:17:PASS:99:594,0",E6,1,LL13-040,left_only,YAL069W,UNK,UNK
1,chrI,1400,CG,C,YAL067W-A,chrI:1400:CG:C:upstream_gene_variant,C:upstream_gene_variant:YAL067W-A/C:upstream_g...,C|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:0,5:5:PASS:99:225,0",O3,1,LL13-040,left_only,YAL067W-A,UNK,UNK
2,chrI,1482,AC,A,YAL067W-A,chrI:1482:AC:A:upstream_gene_variant,A:upstream_gene_variant:YAL067W-A/A:upstream_g...,A|upstream_gene_variant|MODIFIER|YAL067W-A_CDS...,"1:0,17:17:PASS:99:756,0",B4,1,LL13-040,left_only,YAL067W-A,UNK,UNK
3,chrI,2232,AT,A,YAL068C,chrI:2232:AT:A:upstream_gene_variant,A:upstream_gene_variant:YAL068C/A:upstream_gen...,A|upstream_gene_variant|MODIFIER|YAL068C_CDS|G...,"1:1,18:19:PASS:99:765,0",B7,1,LL13-040,left_only,YAL068C,PAU8,seriPAUperin
4,chrI,2232,AT,A,YAL068C,chrI:2232:AT:A:upstream_gene_variant,A:upstream_gene_variant:YAL068C/A:upstream_gen...,A|upstream_gene_variant|MODIFIER|YAL068C_CDS|G...,"1:3,18:21:PASS:99:675,0",C3,1,LL13-040,left_only,YAL068C,PAU8,seriPAUperin
